In [ ]:
!pip install opencv-python numpy tensorflow keras

In [7]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available: 0


In [ ]:
import tensorflow as tf

# Enable GPU memory growth (optional, prevents memory crashes)
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

# Check GPU device
print("Using GPU:", tf.test.is_gpu_available())


In [11]:
import os
import numpy as np
import cv2
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import img_to_array
import pickle

# Define paths
FRAME_DIR = "extracted_frames"  # Change to your actual path
IMAGE_SIZE = (224, 224)
SKIP_FRAMES = 100  # Skip every 100th frame

# Load pre-trained ResNet50 model (without top layer)
base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg')
model = Model(inputs=base_model.input, outputs=base_model.output)

# Function to preprocess and extract features from images
def extract_features():
    features = []
    labels = []
    
    for class_name in os.listdir(FRAME_DIR):
        class_path = os.path.join(FRAME_DIR, class_name)
        
        if os.path.isdir(class_path):  # Check if it's a folder
            print(f"Processing category: {class_name}")
            
            images = sorted(os.listdir(class_path))  # Sort to maintain sequence
            
            for idx, image_name in enumerate(images):
                if idx % SKIP_FRAMES != 0:  # Skip frames
                    continue
                
                image_path = os.path.join(class_path, image_name)
                
                # Read image
                img = cv2.imread(image_path)
                if img is None:
                    continue
                
                img = cv2.resize(img, IMAGE_SIZE)  # Resize to 224x224
                img = img_to_array(img)  # Convert to array
                img = preprocess_input(img)  # Normalize
                
                # Extract features
                img = np.expand_dims(img, axis=0)  # Add batch dimension
                feature = model.predict(img, verbose=0)[0]  # Extract feature
                
                features.append(feature)
                labels.append(class_name)
    
    # Convert lists to numpy arrays
    features = np.array(features)
    labels = np.array(labels)
    
    return features, labels

# Run feature extraction
features, labels = extract_features()

# Save features & labels for future training
with open("features.pkl", "wb") as f:
    pickle.dump((features, labels), f)

print("Feature extraction completed and saved!")


Processing category: Abuse
Processing category: Arrest
Processing category: Arson
Processing category: Assault
Processing category: Burglary
Processing category: Explosion
Processing category: Fighting
Processing category: Normal-Videos-Part-1
Feature extraction completed and saved!


In [13]:
import pickle
from sklearn.model_selection import train_test_split

# Load extracted features
with open("features.pkl", "rb") as f:
    features, labels = pickle.load(f)

# Convert to numpy arrays
import numpy as np
features = np.array(features)
labels = np.array(labels)

# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

print("Data prepared! 🚀")
print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")


Data prepared! 🚀
Training samples: 4517, Testing samples: 1130


In [15]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Train SVM classifier
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)

# Predict on test data
y_pred = svm.predict(X_test)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"SVM Accuracy: {accuracy * 100:.2f}% ✅")


SVM Accuracy: 99.91% ✅


In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.preprocessing import LabelEncoder

# Convert labels to numbers
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

# Build the model with correct input shape
model = Sequential([
    Input(shape=(features.shape[1],)),  # Correct way to define input shape
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(len(set(y_train)), activation='softmax')  # Adjust output size dynamically
])

# Compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, acc = model.evaluate(X_test, y_test)
print(f"🎯 MLP Accuracy: {acc * 100:.2f}% ✅")


Epoch 1/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8504 - loss: 0.4933 - val_accuracy: 0.9973 - val_loss: 0.0086
Epoch 2/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9963 - loss: 0.0138 - val_accuracy: 0.9973 - val_loss: 0.0075
Epoch 3/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9940 - loss: 0.0200 - val_accuracy: 0.9973 - val_loss: 0.0114
Epoch 4/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9974 - loss: 0.0098 - val_accuracy: 0.9982 - val_loss: 0.0042
Epoch 5/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9976 - loss: 0.0057 - val_accuracy: 0.9991 - val_loss: 0.0061
Epoch 6/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9986 - loss: 0.0046 - val_accuracy: 0.9973 - val_loss: 0.0095
Epoch 7/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9965 - loss: 0.0136 - val_accuracy: 0.9991 - val_loss: 0.0109
Epoch 8/10
142/142 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9935 - loss: 0.0180 - val_accuracy: 0.

In [27]:
model.save("trained_model.keras")

In [29]:
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)  # Convert labels to numbers

# Save label mappings
np.save("label_classes.npy", label_encoder.classes_)  # Save label names

In [45]:
import os
import numpy as np
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.preprocessing import LabelEncoder

# Load the trained classification model
model = load_model("trained_model.keras", compile=False)  # Use the correct model format

# Load the label encoder
label_encoder = LabelEncoder()
label_encoder.classes_ = np.load("label_classes.npy", allow_pickle=True)

# Load the same feature extractor (e.g., ResNet50) that was used during training
feature_extractor = ResNet50(weights="imagenet", include_top=False, pooling="avg")

# Path to test images
TEST_FOLDER = "extracted_frames"

# Choose a random category and image
random_category = np.random.choice(os.listdir(TEST_FOLDER))
random_category_path = os.path.join(TEST_FOLDER, random_category)
random_image = np.random.choice(os.listdir(random_category_path))
random_image_path = os.path.join(random_category_path, random_image)

# Load and preprocess the image
img = cv2.imread(random_image_path)
img = cv2.resize(img, (224, 224))
img = preprocess_input(img)
img = np.expand_dims(img, axis=0)  # Add batch dimension

# Extract features using the pre-trained model
features = feature_extractor.predict(img)

# Make a prediction using the classifier
prediction = model.predict(features)
predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])[0]

# Display results
print(f"Actual Category: {random_category}")
print(f"Predicted Category: {predicted_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Actual Category: Fighting
Predicted Category: Fighting


In [49]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Predict class labels
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels

# Compute metrics
accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted')

# Print the results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Print classification report for detailed breakdown
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))


36/36 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
Accuracy: 0.9982
Precision: 0.9982
Recall: 0.9982
F1 Score: 0.9982

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        65
           1       1.00      1.00      1.00       141
           2       1.00      1.00      1.00       241
           3       1.00      0.96      0.98        27
           4       1.00      0.99      1.00       101
           5       1.00      1.00      1.00       275
           6       0.99      1.00      0.99        72
           7       1.00      1.00      1.00       208

    accuracy                           1.00      1130
   macro avg       1.00      0.99      1.00      1130
weighted avg       1.00      1.00      1.00      1130

